In [ ]:
#importing necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path= "sheet2.jpg"
heightImg = 700
widthImg  = 700

#Reading the original image
img=cv2.imread(path)
imgBlank = np.zeros((heightImg,widthImg, 3), np.uint8) # CREATE A BLANK IMAGE FOR TESTING DEBUGGING IF REQUIRED
#imgBigContour = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
#Resizing the image
img=cv2.resize(img,(widthImg,heightImg))

In [ ]:
#Display image using matplotlib
plt.imshow(img)
plt.show()

In [ ]:
#Display image using opencv
cv2.imshow("Original",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
##################################IMAGE PREPROCESSING##################################################################

In [ ]:
#CONVERT IMAGE TO GRAYSCALE
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(imgGray)
plt.title("Grayscale Image")
plt.show()

In [ ]:
#APPLYING GAUSSIAN BLUR
imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1)
plt.imshow(imgBlur)
plt.title("Blurred Image")
plt.show()

In [ ]:
# APPLY CANNY EDGE DETECTION
imgCanny = cv2.Canny(imgBlur,10,70)
plt.imshow(imgCanny)
plt.title("Edges")
plt.show()

In [ ]:
#IMAGE CONTOURING
imgContours = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
contours, hierarchy = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # FIND ALL CONTOURS
cv2.drawContours(imgContours, contours, -100, (0, 255, 0), 10) # DRAW ALL DETECTED CONTOURS
cv2.imshow("contours",imgContours)
cv2.waitKey(0)
cv2.destroyAllWindows()
plt.imshow(imgContours)
plt.title("Contours")
plt.show()

In [ ]:
#function for stacking preprocessed images
def stackImages(imgArray,scale,lables=[]):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
            hor_con[x] = np.concatenate(imgArray[x])
        ver = np.vstack(hor)
        ver_con = np.concatenate(hor)
    else:
        for x in range(0, rows):
            imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        hor_con= np.concatenate(imgArray)
        ver = hor
    if len(lables) != 0:
        eachImgWidth= int(ver.shape[1] / cols)
        eachImgHeight = int(ver.shape[0] / rows)
        #print(eachImgHeight)
        for d in range(0, rows):
            for c in range (0,cols):
                cv2.rectangle(ver,(c*eachImgWidth,eachImgHeight*d),(c*eachImgWidth+len(lables[d][c])*13+27,30+eachImgHeight*d),(255,255,255),cv2.FILLED)
                cv2.putText(ver,lables[d][c],(eachImgWidth*c+10,eachImgHeight*d+20),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,255),2)
    return ver

In [ ]:
#DISPLAYING STACKED IMAGES
imageArray = ([img,imgGray,imgBlur],
              [imgCanny,imgContours,imgBlank])

 # LABELS FOR DISPLAY
lables = [["Original","Gray","Blur"],
           ["Edges","Contours","Blank"]]

stackedImage = stackImages(imageArray,0.5,lables)
cv2.imshow("Result",stackedImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
plt.imshow(stackedImage)
plt.title("RESULT")
plt.show()

In [ ]:
#DETECTING RECTANGLES
def rectContour(contours):
    #max_area = 0
    print("Area :")
    for i in contours:
        area=cv2.contourArea(i)
        print(area)
#PRINTING THE AREA OF THE RECTANGLES
rectCon = rectContour(contours) # FILTER FOR RECTANGLE CONTOURS

In [ ]:
def rectContour(contours):

    rectCon = []
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02 * peri, True)
            print("Corner Points :",approx)
            if len(approx) == 4:
                rectCon.append(i)
    rectCon = sorted(rectCon, key=cv2.contourArea,reverse=True)
    #print(len(rectCon))
    return rectCon

In [ ]:
#PRINTING THE AREA OF THE RECTANGLES
rectCon = rectContour(contours) # FILTER FOR RECTANGLE CONTOURS
#biggestPoints= getCornerPoints(rectCon[0]) # GET CORNER POINTS OF THE BIGGEST RECTANGLE
#gradePoints = getCornerPoints(rectCon[0]) # GET CORNER POINTS OF THE SECOND BIGGEST RECTANGLE
#print(len(rectCon))

In [ ]:
print(len(rectCon))

In [ ]:
#PRINTING THE AREA OF THE RECTANGLES
imgBigContour = img.copy() # COPY IMAGE FOR DISPLAY PURPOSES
rectCon = rectContour(contours) # FILTER FOR RECTANGLE CONTOURS
biggestPoints= getCornerPoints(rectCon[0]) # GET CORNER POINTS OF THE BIGGEST RECTANGLE
gradePoints = getCornerPoints(rectCon[0]) # GET CORNER POINTS OF THE SECOND BIGGEST RECTANGLE
#print(len(rectCon))

In [ ]:
def reorder(myPoints):

    myPoints = myPoints.reshape((4, 2)) # REMOVE EXTRA BRACKET
    print(myPoints)
    myPointsNew = np.zeros((4, 1, 2), np.int32) # NEW MATRIX WITH ARRANGED POINTS
    add = myPoints.sum(1)
    #print(add)
    #print(np.argmax(add))
    myPointsNew[0] = myPoints[np.argmin(add)]  #[0,0]
    myPointsNew[3] =myPoints[np.argmax(add)]   #[w,h]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]  #[w,0]
    myPointsNew[2] = myPoints[np.argmax(diff)] #[h,0]
    #print(diff)
    return myPointsNew


In [ ]:
def getCornerPoints(cont):
    peri = cv2.arcLength(cont, True) # LENGTH OF CONTOUR
    approx = cv2.approxPolyDP(cont, 0.02 * peri, True) # APPROXIMATE THE POLY TO GET CORNER POINTS
    return approx

In [ ]:
def splitBoxes(img):
    rows = np.vsplit(img,5)
    boxes=[]
    for r in rows:
        cols= np.hsplit(r,5)
        for box in cols:
            boxes.append(box)
    return boxes

In [ ]:
biggestPoints= getCornerPoints(rectCon[0]) # GET CORNER POINTS OF THE BIGGEST RECTANGLE
gradePoints = getCornerPoints(rectCon[0]) # GET CORNER POINTS OF THE SECOND BIGGEST RECTANGLE
if biggestPoints.size != 0 and gradePoints.size != 0:

            # BIGGEST RECTANGLE WARPING
            biggestPoints=reorder(biggestPoints) # REORDER FOR WARPING
            cv2.drawContours(imgBigContour, biggestPoints, -1, (0, 255, 0), 20) # DRAW THE BIGGEST CONTOUR
            pts1 = np.float32(biggestPoints) # PREPARE POINTS FOR WARP
            pts2 = np.float32([[0, 0],[widthImg, 0], [0, heightImg],[widthImg, heightImg]]) # PREPARE POINTS FOR WARP
            matrix = cv2.getPerspectiveTransform(pts1, pts2) # GET TRANSFORMATION MATRIX
            imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg)) # APPLY WARP PERSPECTIVE
            
             # SECOND BIGGEST RECTANGLE WARPING
            cv2.drawContours(imgBigContour, gradePoints, -1, (255, 0, 0), 20) # DRAW THE BIGGEST CONTOUR
            gradePoints = reorder(gradePoints) # REORDER FOR WARPING
            ptsG1 = np.float32(gradePoints)  # PREPARE POINTS FOR WARP
            ptsG2 = np.float32([[0, 0], [325, 0], [0, 150], [325, 150]])  # PREPARE POINTS FOR WARP
            matrixG = cv2.getPerspectiveTransform(ptsG1, ptsG2)# GET TRANSFORMATION MATRIX
            imgGradeDisplay = cv2.warpPerspective(img, matrixG, (325, 150)) # APPLY WARP PERSPECTIVE
            
            # APPLY THRESHOLD
            imgWarpGray = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY) # CONVERT TO GRAYSCALE
            imgThresh = cv2.threshold(imgWarpGray, 170, 255,cv2.THRESH_BINARY_INV )[1] # APPLY THRESHOLD AND INVERSE

            boxes = splitBoxes(imgThresh) # GET INDIVIDUAL BOXES
            cv2.imshow("Split Test ", boxes[3])
            countR=0
            countC=0
            myPixelVal = np.zeros((questions,choices)) # TO STORE THE NON ZERO VALUES OF EACH BOX
            for image in boxes:
                #cv2.imshow(str(countR)+str(countC),image)
                totalPixels = cv2.countNonZero(image)
                myPixelVal[countR][countC]= totalPixels
                countC += 1
                if (countC==choices):countC=0;countR +=1

In [ ]:
plt.imshow("Split Test ", boxes[3])
plt.show()